# [Taller 5] Minimos Cuadrados

Interpola los siguientes puntos:

- **p₁** = (5.4, 3.2)
- **p₂ᵢ** = (9.5, 0.7)
- **p₃** = (12.3, -3.6)

> De estos, el punto **p₂ᵢ** debe ser seleccionable y movible.

**Crea un programa que interpole una parábola en tiempo real para los tres puntos.**

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button

# Puntos iniciales
p1 = [5.4, 3.2]
p2 = [9.5, 0.7]
p3 = [12.3, -3.6]

fig, ax = plt.subplots()
plt.title("Interpolación parabólica interactiva")
plt.xlabel("x")
plt.ylabel("y")

# Dibujar p1 y p3 (no movibles)
ax.plot(p1[0], p1[1], 'ro', markersize=8, label="p1")
ax.plot(p3[0], p3[1], 'ro', markersize=8, label="p3")
# Dibujar p2 (movible)
p2_point, = ax.plot(p2[0], p2[1], 'go', markersize=12, label="p2 movible")

# Dibujar parábola inicial
x_parab = np.linspace(min(p1[0], p2[0], p3[0])-1, max(p1[0], p2[0], p3[0])+1, 200)

def parabola(x, y, xp):
    # Interpolación cuadrática: y = ax^2 + bx + c
    A = np.vstack([np.power(x,2), x, np.ones(3)]).T
    a, b, c = np.linalg.solve(A, y)
    return a*xp**2 + b*xp + c

def update_plot():
    # Actualizar punto p2
    p2_point.set_data(p2[0], p2[1])
    # Actualizar parábola
    y_parab = parabola(np.array([p1[0], p2[0], p3[0]]),
                       np.array([p1[1], p2[1], p3[1]]), x_parab)
    parab_line.set_ydata(y_parab)
    fig.canvas.draw_idle()

y_parab = parabola(np.array([p1[0], p2[0], p3[0]]),
                   np.array([p1[1], p2[1], p3[1]]), x_parab)
parab_line, = ax.plot(x_parab, y_parab, 'b-', lw=2)

class DraggablePoint:
    def __init__(self, point, coord):
        self.point = point
        self.coord = coord
        self.press = None
        self.cidpress = point.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.cidrelease = point.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.cidmotion = point.figure.canvas.mpl_connect('motion_notify_event', self.on_motion)

    def on_press(self, event):
        if event.inaxes != self.point.axes: return
        contains, _ = self.point.contains(event)
        if not contains: return
        self.press = (self.coord[0], self.coord[1], event.xdata, event.ydata)

    def on_motion(self, event):
        if self.press is None: return
        if event.inaxes != self.point.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.coord[0] = x0 + dx
        self.coord[1] = y0 + dy
        update_plot()

    def on_release(self, event):
        self.press = None

draggable = DraggablePoint(p2_point, p2)

plt.legend()
plt.show()

RuntimeError: 'widget is not a recognised GUI loop or backend name